---
title: Assignment 04
author:
  - name: Bhargavi Manyala
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
number-sections: true
date: today
date-modified: today
date-format: long
format:
  html:
    theme: cerulean
    toc: true
    toc-depth: 2
  pdf: 
    embed-resources: true
    toc-depth: 2
    geometry: 
      - landscape
      - margin=0.5in
engine: jupyter
jupyter: assignment-04-kernel
execute:
  echo: true
  eval: true
  output: true
  freeze: auto
---



 # Load the Dataset

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np


np.random.seed(42)

pio.renderers.default = "notebook+notebook_connected+vscode"

# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("multiLine", "true")
    .option("escape", "\"")  
    .csv("data/lightcast_job_postings.csv")

)

df.createOrReplaceTempView("job_postings")
#df.show(5)


/home/ubuntu/assignment-04-BhargaviManyala/.venv/lib/python3.12/site-packages/kaleido/__init__.py:14: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/06 00:52:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/06 00:52:21 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


### Missing Value Treatment

In [2]:
# Missing Value Treatment
from pyspark.sql import Window
from pyspark.sql.functions import col, when, isnan, count, expr, median
from pyspark.sql import functions as F

# Calculate overall median salary
overall_median_salarly = df.approxQuantile("SALARY", [0.5], 0.01)[0]

median_by_employment_type = df.groupBy("EMPLOYMENT_TYPE").agg(expr("percentile_approx(SALARY, 0.5)").alias("median_salary_emp_type"))
median_by_employment_type_name = df.groupBy("EMPLOYMENT_TYPE_NAME").agg(expr("percentile_approx(SALARY, 0.5)").alias("median_salary_emp_type_name"))

# Join median values back to the original dataframe
df_salary_imputed = df.join(median_by_employment_type, on="EMPLOYMENT_TYPE", how = "left").join(median_by_employment_type_name, on="EMPLOYMENT_TYPE_NAME", how = "left")


# Replace missing SALARY values
df_salary_imputed=df_salary_imputed.withColumn("SALARY", when(col("SALARY").isNull(), 
                                when (col("median_salary_emp_type").isNotNull(), col("median_salary_emp_type"))
                                .when(col("median_salary_emp_type_name").isNotNull(), col("median_salary_emp_type_name"))
                                .otherwise(overall_median_salarly)
).otherwise(col("SALARY"))) 




# Feature Engineering

### Take columns needed for anlaysis

In [3]:

from pyspark.sql.functions import col, pow
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.types import BooleanType, StringType, IntegerType
from pyspark.sql.functions import regexp_replace, trim

# Drop rows with NA values 
regression_df = df_salary_imputed.dropna(subset=[
    "SALARY", "MIN_YEARS_EXPERIENCE", "MAX_YEARS_EXPERIENCE",
    "EDUCATION_LEVELS_NAME", "EMPLOYMENT_TYPE_NAME", "REMOTE_TYPE_NAME",
    "DURATION", "IS_INTERNSHIP", "COMPANY_IS_STAFFING","median_salary_emp_type", "median_salary_emp_type_name"
]).select(
    "SALARY", "MIN_YEARS_EXPERIENCE", "MAX_YEARS_EXPERIENCE",
    "EDUCATION_LEVELS_NAME", "EMPLOYMENT_TYPE_NAME", "REMOTE_TYPE_NAME",
    "DURATION", "IS_INTERNSHIP", "COMPANY_IS_STAFFING","median_salary_emp_type", "median_salary_emp_type_name"
   
)

# Cast Duration to integer
regression_df = regression_df.withColumn("DURATION", col("DURATION").cast(IntegerType()))



### Clean categorical columns 


In [4]:
# Categorical columns
categorical_cols = [ "EDUCATION_LEVELS_NAME", "EMPLOYMENT_TYPE_NAME", "REMOTE_TYPE_NAME", "IS_INTERNSHIP", "COMPANY_IS_STAFFING"]

# Cast boolean columns to integer
regression_df = regression_df.withColumn("IS_INTERNSHIP", col("IS_INTERNSHIP").cast(IntegerType()))
regression_df = regression_df.withColumn("COMPANY_IS_STAFFING", col("COMPANY_IS_STAFFING").cast(IntegerType()))


# Clean Remote Type Name
regression_df = regression_df.withColumn(
    "REMOTE_TYPE_NAME",
    when(col("REMOTE_TYPE_NAME") == "Remote", "Remote")
    .when(col("REMOTE_TYPE_NAME") == "[None]", "Undefined")
    .when(col("REMOTE_TYPE_NAME") == "Not Remote", "On Premise")
    .when(col("REMOTE_TYPE_NAME") == "Hybrid Remote", "Hybrid")
    .when(col("REMOTE_TYPE_NAME").isNull(), "On Premise")
    .otherwise(col("REMOTE_TYPE_NAME"))
)

# Clean Employment Type Name
regression_df = regression_df.withColumn(
    "EMPLOYMENT_TYPE_NAME",
    when(col("EMPLOYMENT_TYPE_NAME") == "Part-time / full-time", "Flexible")
    .when(col("EMPLOYMENT_TYPE_NAME") == "Part-time (â‰¤ 32 hours)", "Parttime")
    .when(col("EMPLOYMENT_TYPE_NAME") == "Full-time (> 32 hours)", "Fulltime")
    .when(col("EMPLOYMENT_TYPE_NAME").isNull(), "Fulltime")
    .otherwise(col("EMPLOYMENT_TYPE_NAME"))
)

# Clean Education Levels
regression_df = regression_df.withColumn(
    "EDUCATION_LEVELS_NAME",
    trim(regexp_replace(col("EDUCATION_LEVELS_NAME"), r"[\[\]\n]", ""))
)


regression_df.show(5, truncate=False)



+--------+--------------------+--------------------+---------------------+--------------------+----------------+--------+-------------+-------------------+----------------------+---------------------------+
|SALARY  |MIN_YEARS_EXPERIENCE|MAX_YEARS_EXPERIENCE|EDUCATION_LEVELS_NAME|EMPLOYMENT_TYPE_NAME|REMOTE_TYPE_NAME|DURATION|IS_INTERNSHIP|COMPANY_IS_STAFFING|median_salary_emp_type|median_salary_emp_type_name|
+--------+--------------------+--------------------+---------------------+--------------------+----------------+--------+-------------+-------------------+----------------------+---------------------------+
|116500.0|2                   |2                   |"Bachelor's degree"  |Fulltime            |Undefined       |6       |0            |0                  |116500                |116500                     |
|116500.0|7                   |7                   |"No Education Listed"|Fulltime            |Undefined       |18      |0            |1                  |116500           

### Final Features Structure

In [5]:
# Index and One-Hot Encode
indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_idx", handleInvalid="skip") for col in categorical_cols]
encoders = [OneHotEncoder(inputCol=f"{col}_idx", outputCol=f"{col}_vec") for col in categorical_cols]

#Assemble base features
assembler = VectorAssembler(inputCols=["MIN_YEARS_EXPERIENCE", "MAX_YEARS_EXPERIENCE", "DURATION"] + [f"{col}_vec" for col in categorical_cols], outputCol="features")


pipeline = Pipeline(stages=indexers + encoders + [assembler])
regression_data =  pipeline.fit(regression_df).transform(regression_df)
regression_data.select("SALARY","features").show(5, truncate=False)

+--------+-------------------------------------------------------------+
|SALARY  |features                                                     |
+--------+-------------------------------------------------------------+
|116500.0|(28,[0,1,2,3,21,23,26,27],[2.0,2.0,6.0,1.0,1.0,1.0,1.0,1.0]) |
|116500.0|(28,[0,1,2,4,21,23,26],[7.0,7.0,18.0,1.0,1.0,1.0,1.0])       |
|116500.0|(28,[0,1,2,4,21,23,26],[1.0,1.0,8.0,1.0,1.0,1.0,1.0])        |
|116500.0|(28,[0,1,2,3,21,23,26,27],[1.0,1.0,32.0,1.0,1.0,1.0,1.0,1.0])|
|131100.0|(28,[0,1,2,3,21,23,26,27],[2.0,2.0,11.0,1.0,1.0,1.0,1.0,1.0])|
+--------+-------------------------------------------------------------+
only showing top 5 rows



# Train/Test Split

In [6]:
# Split Data
regression_train, regression_test = regression_data.randomSplit([0.8, 0.2], seed=42)

print((regression_data.count(), len(regression_data.columns)))
print((regression_train.count(), len(regression_train.columns)))
print((regression_test.count(), len(regression_test.columns)))

(5039, 22)


(4070, 22)


(969, 22)


# Linear Regression

In [7]:
from pyspark.ml.regression import GeneralizedLinearRegression

feature_names = assembler.getInputCols()

glr = GeneralizedLinearRegression(
    featuresCol="features",
    labelCol="SALARY",
    family="gaussian",
    link="identity",  
    maxIter=10,
    regParam=0.3
)

glr_model = glr.fit(regression_train)
summary = glr_model.summary

### Coefficients and Intercept

In [8]:
# Coefficients and Intercept
print("Intercept: {:.4f}".format(glr_model.intercept))
print("Coefficients:")
for i, coef in enumerate(glr_model.coefficients):
    print(f"Feature {i + 1}: {coef:.4f}")

Intercept: 86792.4416
Coefficients:
Feature 1: 1611.6790
Feature 2: 1611.6790
Feature 3: 28.5445
Feature 4: 1247.2923
Feature 5: 4395.2943
Feature 6: 9613.9357
Feature 7: -25786.3097
Feature 8: 14121.9136
Feature 9: -10802.3368
Feature 10: -1697.8711
Feature 11: 9148.5650
Feature 12: -12465.7919
Feature 13: 1354.9291
Feature 14: -16520.4012
Feature 15: 33093.9741
Feature 16: -9792.1284
Feature 17: -5104.6945
Feature 18: 3404.4202
Feature 19: 5003.3704
Feature 20: -27508.5111
Feature 21: 2140.9095
Feature 22: 6459.6839
Feature 23: -6724.0609
Feature 24: 3701.7084
Feature 25: 7847.9024
Feature 26: 2132.7803
Feature 27: -2242.5622
Feature 28: -36.7908


### Regression Summary

In [9]:
# Summary stats
print("\n--- Regression Summary ---")
print("Coefficient Standard Errors:", [f"{val:.4f}" for val in summary.coefficientStandardErrors])
print("T-Values:", [f"{val:.4f}" for val in summary.tValues])
print("P-Values:", [f"{val:.4f}" for val in summary.pValues])


--- Regression Summary ---


Coefficient Standard Errors: ['25277.8270', '25277.8270', '23.0719', '20920.0566', '20936.4449', '20939.8495', '21079.8038', '21082.5312', '21098.7571', '21110.1720', '21270.4367', '21348.0052', '21575.7584', '21649.7921', '21751.0511', '22067.4219', '22083.8028', '22640.4993', '22402.2783', '25724.6987', '25719.5162', '3159.8576', '3735.1985', '3335.4642', '3406.7472', '3789.8637', '5036.8551', '916.5447', '22025.4149']
T-Values: ['0.0638', '0.0638', '1.2372', '0.0596', '0.2099', '0.4591', '-1.2233', '0.6698', '-0.5120', '-0.0804', '0.4301', '-0.5839', '0.0628', '-0.7631', '1.5215', '-0.4437', '-0.2312', '0.1504', '0.2233', '-1.0693', '0.0832', '2.0443', '-1.8002', '1.1098', '2.3036', '0.5628', '-0.4452', '-0.0401', '3.9406']
P-Values: ['0.9492', '0.9492', '0.2161', '0.9525', '0.8337', '0.6462', '0.2213', '0.5030', '0.6087', '0.9359', '0.6671', '0.5593', '0.9499', '0.4455', '0.1282', '0.6573', '0.8172', '0.8805', '0.8233', '0.2850', '0.9337', '0.0410', '0.0719', '0.2671', '0.0213', '0

### Dispersion Summary

In [10]:
# print(f"\nDispersion: {summary.dispersion:.4f}")
print(f"Null Deviance: {summary.nullDeviance:.4f}")
print(f"Residual DF Null: {summary.residualDegreeOfFreedomNull}")
print(f"Deviance: {summary.deviance:.4f}")
print(f"Residual DF: {summary.residualDegreeOfFreedom}")
print(f"AIC: {summary.aic:.4f}")

Null Deviance: 2321265663379.3027
Residual DF Null: 4069
Deviance: 1808610663869.1785
Residual DF: 4041


AIC: 92652.7396


### Diagnostics

In [11]:
feature_names = summary._call_java("featureNames")


features = ["Intercept"] + list(feature_names)
coefs    = [glr_model.intercept] + list(glr_model.coefficients)
se       = list(summary.coefficientStandardErrors)
tvals    = list(summary.tValues)
pvals    = list(summary.pValues)


print("--- This is a diagnostic check, no need to print in the final doc ---")
print("Length of features:", len(features))
print("Length of coefs:", len(coefs))
print("Length of se:", len(se))
print("Length of tvals:", len(tvals))
print("Length of pvals:", len(pvals))

--- This is a diagnostic check, no need to print in the final doc ---
Length of features: 29
Length of coefs: 29
Length of se: 29
Length of tvals: 29
Length of pvals: 29


### Tabulate

In [12]:
import pandas as pd
from tabulate import tabulate
from IPython.display import HTML

coef_table = pd.DataFrame({
    "Feature": features,
    "Estimate": [f"{v:.4f}" if v is not None else None for v in coefs],
    "Std Error": [f"{v:.4f}" if v is not None else None for v in se],
    "t-stat": [f"{v:.4f}" if v is not None else None for v in tvals],
    "P-Value": [f"{v:.4f}" if v is not None else None for v in pvals]
})


coef_table.to_csv("output/glr_summary.csv", index=False)


HTML(coef_table.to_html())

,Feature,Estimate,Std Error,t-stat,P-Value
0,Intercept,86792.4416,25277.8270,0.0638,0.9492
1,MIN_YEARS_EXPERIENCE,1611.6790,25277.8270,0.0638,0.9492
2,MAX_YEARS_EXPERIENCE,1611.6790,23.0719,1.2372,0.2161
3,DURATION,28.5445,20920.0566,0.0596,0.9525
4,"EDUCATION_LEVELS_NAME_vec_""Bachelor's degree""",1247.2923,20936.4449,0.2099,0.8337
5,"EDUCATION_LEVELS_NAME_vec_""No Education Listed""",4395.2943,20939.8495,0.4591,0.6462
6,"EDUCATION_LEVELS_NAME_vec_""Bachelor's degree"", ""Master's degree""",9613.9357,21079.8038,-1.2233,0.2213
7,"EDUCATION_LEVELS_NAME_vec_""High school or GED"", ""Bachelor's degree""",-25786.3097,21082.5312,0.6698,0.5030
8,"EDUCATION_LEVELS_NAME_vec_""Bachelor's degree"", ""Master's degree"", ""Ph.D. or professional degree""",14121.9136,21098.7571,-0.5120,0.6087
9,"EDUCATION_LEVELS_NAME_vec_""High school or GED""",-10802.3368,21110.1720,-0.0804,0.9359


# Polynomial Regression

In [13]:
# Create squared term for Polynomial Regression
poly_data = regression_data.withColumn(
    "MAX_YEARS_EXPERIENCE_SQ",
    pow(col("MAX_YEARS_EXPERIENCE"), 2)
)

# Assemble polynomial features
assembler_poly = VectorAssembler(
    inputCols=[
        "MIN_YEARS_EXPERIENCE", "MAX_YEARS_EXPERIENCE_SQ", "MAX_YEARS_EXPERIENCE",
        "DURATION"
    ] + [f"{col}_vec" for col in categorical_cols],
    outputCol="features_poly"
)

poly_data = assembler_poly.transform(poly_data)

poly_data.select("SALARY", "features_poly").show(5, truncate=False)

+--------+-------------------------------------------------------------------+
|SALARY  |features_poly                                                      |
+--------+-------------------------------------------------------------------+
|116500.0|(29,[0,1,2,3,4,22,24,27,28],[2.0,4.0,2.0,6.0,1.0,1.0,1.0,1.0,1.0]) |
|116500.0|(29,[0,1,2,3,5,22,24,27],[7.0,49.0,7.0,18.0,1.0,1.0,1.0,1.0])      |
|116500.0|(29,[0,1,2,3,5,22,24,27],[1.0,1.0,1.0,8.0,1.0,1.0,1.0,1.0])        |
|116500.0|(29,[0,1,2,3,4,22,24,27,28],[1.0,1.0,1.0,32.0,1.0,1.0,1.0,1.0,1.0])|
|131100.0|(29,[0,1,2,3,4,22,24,27,28],[2.0,4.0,2.0,11.0,1.0,1.0,1.0,1.0,1.0])|
+--------+-------------------------------------------------------------------+
only showing top 5 rows



### Split Data

In [ ]:

polyreg_train, polyreg_test = poly_data.randomSplit([0.8, 0.2], seed=42)

print((poly_data.count(), len(poly_data.columns)))
print((polyreg_train.count(), len(polyreg_train.columns)))
print((polyreg_test.count(), len(polyreg_test.columns)))

(5039, 24)


(4070, 24)


(969, 24)


In [15]:
from pyspark.ml.regression import GeneralizedLinearRegression

feature_names = assembler.getInputCols()

poly_glr_max_years = GeneralizedLinearRegression(
    featuresCol="features_poly",
    labelCol="SALARY",
    family="gaussian",   
    link="identity",     
    maxIter=10,         
    regParam=0.3         
)

poly_glr_max_years_model = poly_glr_max_years.fit(poly_data)
poly_summary = poly_glr_max_years_model.summary

### Coefficients and Intercept

In [16]:
print("Intercept: {:.4f}".format(poly_glr_max_years_model.intercept))
print("Coefficients:")
for i, coef in enumerate(poly_glr_max_years_model.coefficients):
    print(f"Feature {i + 1}: {coef:.4f}")

Intercept: 87729.4985
Coefficients:
Feature 1: 3073.9396
Feature 2: -292.0763
Feature 3: 3073.9396
Feature 4: 29.5828
Feature 5: -2253.9775
Feature 6: 805.2130
Feature 7: 5881.4501
Feature 8: -30439.8972
Feature 9: 8412.8920
Feature 10: -13680.3546
Feature 11: -4660.4367
Feature 12: 7967.9560
Feature 13: -14214.1229
Feature 14: -2857.2642
Feature 15: -15252.4114
Feature 16: 38851.8773
Feature 17: -12908.6915
Feature 18: -9159.5725
Feature 19: 13889.7107
Feature 20: 4123.7347
Feature 21: -40022.9996
Feature 22: 27224.9060
Feature 23: 5759.7671
Feature 24: -6588.6707
Feature 25: 3972.6485
Feature 26: 8353.6157
Feature 27: 2403.5211
Feature 28: -4220.3464
Feature 29: 214.4410


### Ploynomial Summary

In [18]:

print("\n--- Poly Summary ---")
print("Coefficient Standard Errors:", [f"{val:.4f}" for val in poly_summary.coefficientStandardErrors])
print("T-Values:", [f"{val:.4f}" for val in poly_summary.tValues])
print("P-Values:", [f"{val:.4f}" for val in poly_summary.pValues])


--- Poly Summary ---


Coefficient Standard Errors: ['22747.6805', '36.5365', '22747.6805', '20.5955', '20846.6950', '20859.1237', '20863.3147', '20971.9940', '20987.9155', '20990.0817', '21019.8840', '21100.5976', '21203.5634', '21380.9216', '21419.1586', '21567.5225', '21793.5925', '21905.9591', '22009.1299', '22150.2630', '24166.2009', '24163.3570', '3057.6640', '3566.5832', '2907.7548', '2969.5007', '3355.2443', '4582.4088', '821.4479', '21780.4771']
T-Values: ['0.1351', '-7.9941', '0.1351', '1.4364', '-0.1081', '0.0386', '0.2819', '-1.4515', '0.4008', '-0.6518', '-0.2217', '0.3776', '-0.6704', '-0.1336', '-0.7121', '1.8014', '-0.5923', '-0.4181', '0.6311', '0.1862', '-1.6562', '1.1267', '1.8837', '-1.8473', '1.3662', '2.8131', '0.7163', '-0.9210', '0.2611', '4.0279']
P-Values: ['0.8925', '0.0000', '0.8925', '0.1510', '0.9139', '0.9692', '0.7780', '0.1467', '0.6886', '0.5146', '0.8245', '0.7057', '0.5027', '0.8937', '0.4764', '0.0717', '0.5537', '0.6759', '0.5280', '0.8523', '0.0978', '0.2599', '0.0597',

### Dispersion Summary

In [19]:
# print(f"\nDispersion: {summary.dispersion:.4f}")
print(f"Null Deviance: {poly_summary.nullDeviance:.4f}")
print(f"Residual DF Null: {poly_summary.residualDegreeOfFreedomNull}")
print(f"Deviance: {poly_summary.deviance:.4f}")
print(f"Residual DF: {poly_summary.residualDegreeOfFreedom}")
print(f"AIC: {poly_summary.aic:.4f}")

Null Deviance: 2909123659016.3213
Residual DF Null: 5038
Deviance: 2244317022136.5864
Residual DF: 5009


AIC: 114711.0237


### Diagnostics

In [20]:
feature_names = poly_summary._call_java("featureNames")


poly_features = ["Intercept"] + list(feature_names)
poly_coefs    = [poly_glr_max_years_model.intercept] + list(poly_glr_max_years_model.coefficients)
poly_se       = list(poly_summary.coefficientStandardErrors)
poly_tvals    = list(poly_summary.tValues)
poly_pvals    = list(poly_summary.pValues)


print("--- This is a diagnostic check, no need to print in the final doc ---")
print("Length of features:", len(poly_features))
print("Length of coefs:", len(poly_coefs))
print("Length of se:", len(poly_se))
print("Length of tvals:", len(poly_tvals))
print("Length of pvals:", len(poly_pvals))

--- This is a diagnostic check, no need to print in the final doc ---
Length of features: 30
Length of coefs: 30
Length of se: 30
Length of tvals: 30
Length of pvals: 30


### Tabulate

In [ ]:
import pandas as pd
from tabulate import tabulate
from IPython.display import HTML

poly_coef_table = pd.DataFrame({
    "Feature": poly_features,
    "Estimate": [f"{v:.4f}" if v is not None else None for v in poly_coefs],
    "Std Error": [f"{v:.4f}" if v is not None else None for v in poly_se],
    "t-stat": [f"{v:.4f}" if v is not None else None for v in poly_tvals],
    "P-Value": [f"{v:.4f}" if v is not None else None for v in poly_pvals]
})


poly_coef_table.to_csv("output/poly_summary.csv", index=False)


HTML(poly_coef_table.to_html())